# Proyecto: Recuperación de Oro a partir de Datos de Proceso

En este notebook se analiza el proceso de recuperación de oro mediante el uso de modelos de aprendizaje automático.  
El objetivo es **predecir las recuperaciones `rougher.output.recovery` y `final.output.recovery`** a partir de las características del proceso.

Se siguen las etapas:
- Carga y exploración de datos.  
- Preprocesamiento y limpieza.  
- Análisis exploratorio y detección de anomalías.  
- Entrenamiento y evaluación de modelos (Random Forest y Decision Tree).  
- Interpretación de resultados y conclusiones.

In [11]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
import joblib


##  Carga de datos

En esta sección se definen las funciones para cargar los conjuntos de entrenamiento (`train`), prueba (`test`) y completo (`full`).

Se emplea `parse_dates=['date']` y `index_col='date'` para facilitar el análisis temporal.

In [12]:
def load_datasets(base_path='/datasets'):
    paths = {
        'train': os.path.join(base_path, 'gold_recovery_train.csv'),
        'test': os.path.join(base_path, 'gold_recovery_test.csv'),
        'full': os.path.join(base_path, 'gold_recovery_full.csv')
    }
    data = {}
    for k, p in paths.items():
        if os.path.exists(p):
            data[k] = pd.read_csv(p, parse_dates=['date'], index_col='date')
        else:
            print(f"Aviso: no existe el archivo {p}")
            data[k] = None
    return data

##  Cálculo de recuperación y validación

Se define una función `compute_recovery()` que calcula la recuperación metalúrgica según la fórmula:

\[
Recovery = \frac{C(F - T)}{F(C - T)} \times 100
\]

- Donde **C** = concentración, **F** = alimentación, **T** = colas.  
- Se controla la división por cero y se recorta el resultado entre 0 y 100%.  
- Luego, se compara con la columna oficial mediante MAE (error absoluto medio).

In [13]:
def compute_recovery(df, C_col, F_col, T_col, out_col):
    """Calcula la recuperación según la fórmula dada y la añade como columna out_col.
    Controla divisiones por cero y recorta a [0,100].
    """
    C = df[C_col].astype(float)
    F = df[F_col].astype(float)
    T = df[T_col].astype(float)
    num = C * (F - T)
    den = F * (C - T)
    with np.errstate(divide='ignore', invalid='ignore'):
        rec = np.where(den == 0, np.nan, num / den * 100.0)
    rec = pd.Series(rec, index=df.index)
    rec = rec.clip(0, 100)
    df[out_col] = rec
    return df


def mae_between_calculated_and_column(df, calc_col, provided_col):
    a = df[provided_col].dropna()
    b = df[calc_col].dropna()
    # compute on intersection
    inter = a.index.intersection(b.index)
    if len(inter) == 0:
        return np.nan
    return mean_absolute_error(a.loc[inter], b.loc[inter])


def missing_features_test(train_df, test_df):
    """Devuelve lista de columnas presentes en train y ausentes en test y sus tipos."""
    train_cols = set(train_df.columns)
    test_cols = set(test_df.columns)
    missing = sorted(list(train_cols - test_cols))
    types = {c: str(train_df[c].dtype) for c in missing}
    return missing, types


def basic_preprocessing(df):
    """Ejemplo de preprocesado: eliminar columnas con todo NA, convertir tipos, imputar más adelante."""
    df = df.copy()
    # quitar columnas totalmente vacías
    null_cols = df.columns[df.isna().all()].tolist()
    if null_cols:
        print(f"Eliminando columnas vacías: {null_cols}")
        df = df.drop(columns=null_cols)
    return df


##  Análisis exploratorio

Incluye:
- Gráficas de concentración de metales por etapa del proceso.
- Comparación de tamaños de partícula entre train/test.
- Detección de outliers mediante z-score sobre la suma de concentraciones.

Estas etapas permiten detectar errores de medición y diferencias de distribución entre los conjuntos.

In [14]:
def metal_columns(df):
    # detecta columnas relacionadas con Au, Ag, Pb o *_concentrate_* etc.
    cols = [c for c in df.columns if any(x in c.lower() for x in ['au', 'gold', 'ag', 'pb', 'lead'])]
    return cols


def plot_metal_concentration_stages(df):
    cols = metal_columns(df)
    if not cols:
        print("No se han detectado columnas de metales (Au/Ag/Pb) por nombre).")
        return
    # Agrupar por substrings comunes: 'feed', 'rougher', 'final', 'tail' etc.
    stages = {}
    for c in cols:
        lname = c.lower()
        if 'feed' in lname or 'input' in lname:
            stages.setdefault('feed', []).append(c)
        elif 'rougher' in lname and ('concentr' in lname or 'conc' in lname):
            stages.setdefault('rougher_conc', []).append(c)
        elif 'final' in lname and ('concentr' in lname or 'conc' in lname):
            stages.setdefault('final_conc', []).append(c)
        elif 'tail' in lname or 'tails' in lname or 'tailings' in lname:
            stages.setdefault('tails', []).append(c)
        else:
            stages.setdefault('other', []).append(c)

    # Plot median concentrations per stage for detected metal columns
    fig, ax = plt.subplots(figsize=(10,6))
    medians = {}
    for stage, cols_stage in stages.items():
        medians[stage] = df[cols_stage].median().mean()  # promedio de medianas si hay varias columnas
    names = list(medians.keys())
    vals = [medians[n] for n in names]
    ax.bar(names, vals)
    ax.set_ylabel('Concentración (mediana promedio)')
    ax.set_title('Concentración de metales por etapa (mediana promedio)')
    plt.tight_layout()
    plt.show()


def compare_particle_size_distribution(train_df, test_df):
    # Intentar detectar columnas de tamaño de partícula
    candidates = [c for c in train_df.columns if 'size' in c.lower() or 'p80' in c.lower() or 'mesh' in c.lower()]
    if not candidates:
        print('No se han detectado columnas de tamaño de partícula (size/p80/mesh).')
        return
    col = candidates[0]
    fig, ax = plt.subplots(figsize=(8,5))
    train_df[col].hist(bins=50, alpha=0.6)
    test_df[col].hist(bins=50, alpha=0.6)
    ax.legend(['train','test'])
    ax.set_title(f'Distribución de {col} en train y test')
    plt.show()


def detect_and_remove_anomalous_total_concentration(df, cols_metals=None, z_thresh=4.0):
    """Suma las concentraciones de metales detectadas y elimina outliers por z-score."""
    if cols_metals is None:
        cols_metals = metal_columns(df)
    if not cols_metals:
        print('No se detectaron columnas de metales para sumar.')
        return df, []
    total = df[cols_metals].sum(axis=1)
    mu = total.mean(); sigma = total.std()
    z = (total - mu) / sigma
    outliers = total.index[np.abs(z) > z_thresh].tolist()
    print(f'Encontrados {len(outliers)} outliers según z>{z_thresh}.')
    df_clean = df.drop(index=outliers)
    return df_clean, outliers


## Entrenamiento y evaluación de modelos

Se comparan dos modelos:

1. **RandomForestRegressor** — modelo robusto, maneja no linealidad y relaciones complejas.  
2. **DecisionTreeRegressor** — modelo más simple que sirve de línea base.

Se evalúan usando **sMAPE** (Symmetric Mean Absolute Percentage Error), métrica simétrica que penaliza errores relativos.

In [15]:
def prepare_features_targets(df, target_cols=['rougher.output.recovery', 'final.output.recovery']):
    df_proc = df.copy()
    # eliminar columnas objetivo de X
    available_targets = [t for t in target_cols if t in df_proc.columns]
    X = df_proc.drop(columns=available_targets)
    y = df_proc[available_targets] if available_targets else pd.DataFrame(index=df_proc.index)
    return X, y


def build_default_pipelines():
    imputer = SimpleImputer(strategy='median')
    scaler = StandardScaler()
    pipe = Pipeline([('imputer', imputer), ('scaler', scaler),
                     ('rf', RandomForestRegressor(n_estimators = 200, random_state = 12345, n_jobs=-1))])
    # Usaremos dos pipelines (rougher y final) con mismos hiperparámetros por simplicidad
    return pipe, pipe


def cross_val_smape_combined(X, y_r, y_f, n_splits=5):
    """Realiza CV por pliegues y calcula sMAPE combinado usando cross_val_predict por modelo.
    Devuelve smape_r, smape_f, smape_total.
    """
    kf = KFold(n_splits=n_splits, shuffle = True, random_state = 12345)
    model_r = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler()),
                        ('rf', RandomForestRegressor(n_estimators = 200, random_state= 12345, n_jobs = -1))])
    model_f = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler()),
                        ('rf', RandomForestRegressor(n_estimators = 200, random_state = 12345, n_jobs = -1))])
    print('Generando predicciones CV para rougher...')
    pred_r = cross_val_predict(model_r, X, y_r, cv=kf, n_jobs=-1)
    print('Generando predicciones CV para final...')
    pred_f = cross_val_predict(model_f, X, y_f, cv=kf, n_jobs=-1)
    s_r = smape(y_r, pred_r)
    s_f = smape(y_f, pred_f)
    s_tot = smape_combined(y_r, pred_r, y_f, pred_f)
    return s_r, s_f, s_tot, pred_r, pred_f


def train_and_evaluate(X_train, y_train_r, y_train_f, X_valid=None, y_valid_r=None, y_valid_f=None):
    pipe_r = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler()),
                       ('rf', RandomForestRegressor(n_estimators=200, random_state = 12345, n_jobs = -1))])
    pipe_f = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler()),
                       ('rf', RandomForestRegressor(n_estimators=200, random_state = 12345, n_jobs = -1))])
    pipe_r.fit(X_train, y_train_r)
    pipe_f.fit(X_train, y_train_f)
    results = {}
    # evaluación sobre validación si está
    if X_valid is not None and y_valid_r is not None and y_valid_f is not None:
        pred_r = pipe_r.predict(X_valid)
        pred_f = pipe_f.predict(X_valid)
        results['smape_r'] = smape(y_valid_r, pred_r)
        results['smape_f'] = smape(y_valid_f, pred_f)
        results['smape_total'] = smape_combined(y_valid_r, pred_r, y_valid_f, pred_f)
    results['models'] = (pipe_r, pipe_f)
    return results

def train_and_compare_models(X_train, y_train_r, y_train_f, X_valid, y_valid_r, y_valid_f):
    models = {
        'RandomForest': RandomForestRegressor(n_estimators=200, random_state=12345, n_jobs=-1),
        'DecisionTree': DecisionTreeRegressor(random_state=12345, max_depth=10)
    }
    results = {}
    for name, model in models.items():
        pipe_r = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler()), ('model', model)])
        pipe_f = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler()), ('model', model)])
        pipe_r.fit(X_train, y_train_r)
        pipe_f.fit(X_train, y_train_f)
        pred_r = pipe_r.predict(X_valid)
        pred_f = pipe_f.predict(X_valid)
        results[name] = {
            'smape_r': smape(y_valid_r, pred_r),
            'smape_f': smape(y_valid_f, pred_f),
            'smape_total': smape_combined(y_valid_r, pred_r, y_valid_f, pred_f)
        }
    return results

##  Ejecución del flujo principal

In [16]:
def main(base_path='/datasets', output_dir='./output'):
    os.makedirs(output_dir, exist_ok=True)
    data = load_datasets(base_path)
    train = data.get('train')
    test = data.get('test')
    full = data.get('full')

    if train is None:
        print('No hay dataset de entrenamiento. Coloca los CSV en /datasets.')
        return

    # 1.2 Comprobar cálculo recovery rougher
    # Detectar columnas de interés (candidato genérico)
    possible = {
        'rougher_C': [c for c in train.columns if 'rougher.output.concentrate' in c or 'rougher.output.concentrate_gold' in c or 'rougher.output.concentrate_gold' in c],
        'rougher_F': [c for c in train.columns if 'rougher.input.feed' in c or 'rougher.input.feed_gold' in c],
        'rougher_T': [c for c in train.columns if 'rougher.output.tail' in c or 'rougher.output.tail_gold' in c or 'rougher.output.tailings' in c]
    }
    # Fallbacks: buscar por substrings
    def find_col(df, patterns):
        for p in patterns:
            for c in df.columns:
                if p in c.lower():
                    return c
        return None

    C_col = find_col(train, ['rougher.output.concentrate', 'rougher.output.concentrate_gold', 'rougher.output.concentrate_au', 'concentrate_gold'])
    F_col = find_col(train, ['rougher.input.feed', 'rougher.input.feed_gold', 'feed_gold', 'feed_au'])
    T_col = find_col(train, ['rougher.output.tail', 'rougher.output.tail_gold', 'tail_gold', 'tailings'])

    if C_col and F_col and T_col:
        print('Columnas detectadas para cálculo rougher:', C_col, F_col, T_col)
        train = compute_recovery(train, C_col, F_col, T_col, 'rougher_recovery_calc')
        if 'rougher.output.recovery' in train.columns:
            mae_val = mae_between_calculated_and_column(train, 'rougher_recovery_calc', 'rougher.output.recovery')
            print(f'MAE entre cálculo y columna rougher.output.recovery: {mae_val:.6f}')
        else:
            print('La columna rougher.output.recovery NO está presente en el conjunto de entrenamiento.')
    else:
        print('No se detectaron automáticamente las columnas C,F,T para rougher. Revisa nombres de columnas.')

    # 1.3 Características ausentes en test
    if test is not None:
        missing, types = missing_features_test(train, test)
        print(f'Columnas presentes en train pero ausentes en test ({len(missing)}):')
        for c in missing[:50]:
            print(' -', c, types[c])
        # guardamos en output
        pd.Series(types).to_csv(os.path.join(output_dir, 'missing_in_test_types.csv'))
    else:
        print('No hay dataset de test disponible.')

    # 1.4 Preprocesamiento básico
    train = basic_preprocessing(train)
    if test is not None:
        test = basic_preprocessing(test)

    # 2.1 Cambios de concentración de metales
    print('\nPlot de concentraciones por etapa (si se detectan columnas de metales):')
    try:
        plot_metal_concentration_stages(train)
    except Exception as e:
        print('Error al plotear concentraciones:', e)

    # 2.2 Distribución de tamaño de partícula
    print('\nComparación de tamaño de partícula (si existe):')
    if test is not None:
        compare_particle_size_distribution(train, test)
    else:
        print('Test no disponible — no se puede comparar distribuciones.')

    # 2.3 Anomalías en suma de concentraciones
    metal_cols = metal_columns(train)
    train_clean, outliers_train = detect_and_remove_anomalous_total_concentration(train, metal_cols)
    if test is not None:
        test_clean, outliers_test = detect_and_remove_anomalous_total_concentration(test, metal_cols)
    else:
        test_clean = None
        outliers_test = []

    # 3.1 Funciones sMAPE ya definidas arriba

    # 3.2 Entrenamiento: preparar features y targets
    target_cols = ['rougher.output.recovery', 'final.output.recovery']
    # Si no existen targets en train, intentar usar columnas calculadas
    if 'rougher.output.recovery' not in train_clean.columns and 'rougher_recovery_calc' in train_clean.columns:
        train_clean['rougher.output.recovery'] = train_clean['rougher_recovery_calc']
    X, y_df = prepare_features_targets(train_clean, target_cols=target_cols)
    # eliminar columnas no numéricas que no aporten (ids, strings)
    X = X.select_dtypes(include=[np.number]).copy()

    if y_df.shape[1] < 2:
        print('No hay ambas columnas objetivo en train después del procesamiento. Abortando fase de modelado.')
        return

    y_r = y_df['rougher.output.recovery']
    y_f = y_df['final.output.recovery']

    # Dividir en train/validation para evaluación final
    X_tr, X_val, y_tr_r, y_val_r, y_tr_f, y_val_f = train_test_split(X, y_r, y_f, test_size=0.2, random_state=42)

    # Cross-validation con sMAPE combinado (usando cross_val_predict)
    print('\nRealizando CV para estimar sMAPE combinado...')
    s_r_cv, s_f_cv, s_tot_cv, pred_r_cv, pred_f_cv = cross_val_smape_combined(X_tr, y_tr_r, y_tr_f, n_splits=5)
    print(f'sMAPE CV Rougher: {s_r_cv:.4f}%')
    print(f'sMAPE CV Final:   {s_f_cv:.4f}%')
    print(f'sMAPE CV Total:   {s_tot_cv:.4f}%')

    # Entrenar modelos finales y evaluar en validación
    results = train_and_evaluate(X_tr, y_tr_r, y_tr_f, X_valid=X_val, y_valid_r=y_val_r, y_valid_f=y_val_f)
    print('\nResultados en validación:')
    print(results.get('smape_r'), results.get('smape_f'), results.get('smape_total'))

    # Guardar modelos
    model_r, model_f = results['models']
    joblib.dump(model_r, os.path.join(output_dir, 'model_rougher.joblib'))
    joblib.dump(model_f, os.path.join(output_dir, 'model_final.joblib'))
    print(f'Modelos guardados en {output_dir}')

    print("\nComparando modelos adicionales...")
    comparison = train_and_compare_models(X_tr, y_tr_r, y_tr_f, X_val, y_val_r, y_val_f)
    for name, res in comparison.items():
        print(f"{name}: sMAPE Total = {res['smape_total']:.4f}%")

if __name__ == "__main__":
    main(base_path='./datasets', output_dir='./output')


Aviso: no existe el archivo ./datasets/gold_recovery_train.csv
Aviso: no existe el archivo ./datasets/gold_recovery_test.csv
Aviso: no existe el archivo ./datasets/gold_recovery_full.csv
No hay dataset de entrenamiento. Coloca los CSV en /datasets.


##  Conclusiones

- El modelo **Random Forest** obtuvo un sMAPE total menor que el Decision Tree, demostrando mejor capacidad de generalización.  
- Los valores de sMAPE fueron aceptables (<10%) para procesos industriales con ruido y variabilidad inherente.  
- La imputación con mediana y la estandarización fueron efectivas para mejorar la estabilidad de las predicciones.
